In [1]:
import os
from tqdm import tqdm

import config
import modules.dataloaders as detection_data_loader
import modules.metrics as metrics

import numpy as np
import torch
import torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
import onnx
import onnxruntime

# Check Models

In [3]:
brevitas_no_comp_name = './onnx_models/w4W2h8b4__bed_detector___no_comp__fixed_point__qcdq__CPU.onnx'
brevitas_no_comp_model = onnx.load(brevitas_no_comp_name)
onnx.checker.check_model(brevitas_no_comp_model)

In [4]:
brevitas_aimet_name = './onnx_models/w5W4a8b5__bed_detector___aimet__fixed_point__qcdq__CPU.onnx'
brevitas_aimet_model = onnx.load(brevitas_aimet_name)
onnx.checker.check_model(brevitas_aimet_model)

In [5]:
brevitas_aimet_cpu_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx'
brevitas_aimet_cpu_model = onnx.load(brevitas_aimet_cpu_name)
onnx.checker.check_model(brevitas_aimet_cpu_name)

# Helper function to convert pytorch tensors to numpy. Useful to handle datatset output.

In [6]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Detection Loaders

In [7]:
detection_loader = detection_data_loader.get_val_loader()
detection_dfire_loader = detection_data_loader.get_dfire_val_loader()
detection_fasdd_uav_loader = detection_data_loader.get_fasdd_uav_val_loader()
detection_fasdd_cv_loader = detection_data_loader.get_fasdd_cv_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154

TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to

# Evaluate Detector

In [8]:
map_metric = MeanAveragePrecision(
    box_format='xyxy',
    iou_thresholds=[config.IOU_THRESHOLD],
    class_metrics=True, # Enables separated metrics for each class
    #average='micro',
    extended_summary=False).to('cpu')

### Evaluation Function

In [9]:
def eval_detector_onnx(loader, model_name, score_thres):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            out = ort_session.run(None, ort_inputs)
            
            # out of onnx session is a list: [out_tensor with batch dim] -> [ (1,12,7,7) ] 
            # -> out[0] = (1,12,7,7)
            # -> out[0][0] = (12,7,7)
            #print(f'Out type: {type(out[0])} - Output shape: {out[0].shape}')
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)
            #print(f'Out shape after permute: {out.shape}')
            #print(f'Out shape after indexing: {out[0].shape}')
            
            # Label should be [xc, yc, w, h, score=1, smoke, fire] in 7x7 square
            #print(f'Label indexed shape: {label[idx].shape}')
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"].item():.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

In [10]:
print("\n________________________________ Brevitas NO Comp: FULL _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    brevitas_no_comp_name,
    0.2
)
print("\n________________________________ Brevitas NO Comp: DFire _______________________________")
_ = eval_detector_onnx(
    detection_dfire_loader, 
    brevitas_no_comp_name,
    0.2
)
print("\n________________________________ Brevitas NO Comp: FASDD UAV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_uav_loader, 
    brevitas_no_comp_name,
    0.2
)
print("\n________________________________ Brevitas NO Comp: FASDD CV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_cv_loader, 
    brevitas_no_comp_name,
    0.2
)
print("\n________________________________ Brevitas AIMET FPGA: FULL _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    brevitas_aimet_name,
    0.2
)
print("\n________________________________ Brevitas AIMET FPGA: DFire _______________________________")
_ = eval_detector_onnx(
    detection_dfire_loader, 
    brevitas_aimet_name,
    0.2
)
print("\n________________________________ Brevitas AIMET FPGA: FASDD UAV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_uav_loader, 
    brevitas_aimet_name,
    0.2
)
print("\n________________________________ Brevitas AIMET FPGA: FASDD CV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_cv_loader, 
    brevitas_aimet_name,
    0.2
)
print("\n________________________________ Brevitas AIMET CPU: FULL _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    brevitas_aimet_cpu_name,
    0.2
)
print("\n________________________________ Brevitas AIMET CPU: DFire _______________________________")
_ = eval_detector_onnx(
    detection_dfire_loader, 
    brevitas_aimet_cpu_name,
    0.2
)
print("\n________________________________ Brevitas AIMET CPU: FASDD UAV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_uav_loader, 
    brevitas_aimet_cpu_name,
    0.2
)
print("\n________________________________ Brevitas AIMET CPU: FASDD CV _______________________________")
_ = eval_detector_onnx(
    detection_fasdd_cv_loader, 
    brevitas_aimet_cpu_name,
    0.2
)


________________________________ Brevitas NO Comp: FULL _______________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 361/361 [12:36<00:00,  2.10s/it]


Smoke -> AP: 0.6230 - AR: 0.6823
Fire -> AP: 0.6087 - AR: 0.6503
mAP: 0.6159

________________________________ Brevitas NO Comp: DFire _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [02:13<00:00,  2.16s/it]


Smoke -> AP: 0.4124 - AR: 0.5269
Fire -> AP: 0.3196 - AR: 0.4007
mAP: 0.3660

________________________________ Brevitas NO Comp: FASDD UAV _______________________________


...

Smoke -> AP: 0.4477 - AR: 0.5630
Fire -> AP: 0.3346 - AR: 0.4315
mAP: 0.3911

________________________________ Brevitas AIMET CPU: FASDD UAV _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:57<00:00,  2.05s/it]


Smoke -> AP: 0.6586 - AR: 0.6850
Fire -> AP: 0.5425 - AR: 0.5525
mAP: 0.6006

________________________________ Brevitas AIMET CPU: FASDD CV _______________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████| 242/242 [07:26<00:00,  1.84s/it]


Smoke -> AP: 0.6831 - AR: 0.7477
Fire -> AP: 0.6937 - AR: 0.7526
mAP: 0.6884


# Aladdin Metrics to Get Precision and Precision-Recall Curves

In [11]:
import modules.metrics_mAP_aladdin as aladdin_metrics

In [12]:
''' ============================
    Cell to Box Mask
============================ '''
cell2box_mask = torch.zeros((config.S, config.S, 2))
for i in range(config.S):
    for j in range(config.S):
        cell2box_mask[i,j,0] = j
        cell2box_mask[i,j,1] = i  

### Aladdin mAP modified for ONNX detection with thresholds

In [13]:
def aladdin_get_bboxes(
    loader,
    detector_model_name,
    S=config.S,
    B=config.B,
    C=config.C,
    mask=cell2box_mask,
    device='cpu',
    iou_threshold=0.5,
    score_thres=0.2,
    box_format="midpoint"):
    
    '''
    
    Return:
        - all_pred_boxes
        - all_true_boxes
        Format: [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
    '''
    
    detect_session = onnxruntime.InferenceSession(detector_model_name, providers=["CPUExecutionProvider"])
    
    
    all_pred_boxes = []
    all_true_boxes = []

    # Original Code
    # make sure model is in eval before get bboxes
    #model.eval()
    train_idx = 0

    loop = tqdm(loader, desc='Get Boxes', leave=True)
    for batch_idx, (imgs, labels) in enumerate(loop):
        # Original Code
        # Inference in GPU. Move tensor to CPU in outcell_2_outboxes
        # imgs = imgs.to(device)
        # labels = labels.to(device)


        for idx in range(config.BATCH_SIZE):
            
            detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(imgs[idx].unsqueeze(dim=0))}
            out = detect_session.run(None, detect_inputs)
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)         
            
            # Original Code
            # with torch.no_grad():
            #     predictions = model(imgs)

            # Original Code
            # Remove Permute from the model
            #predictions = predictions.permute(0, 2, 3, 1) # Original Code

            #batch_size = imgs.shape[0] # Original Code

            true_bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=labels[idx].unsqueeze(dim=0), 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=False)
            bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=out, 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=True)

            for idx in range(1): # Only 1 image every time, due to ONNX prediction in CPU
                nms_boxes = aladdin_metrics.nms_yv1_getBBoxes(
                    bboxes[idx],
                    iou_threshold=iou_threshold,
                    threshold=score_thres,
                    box_format=box_format, # Midpoint, to use iou_tensor inside
                )


                # Plot some examples
                #if batch_idx == 0 and idx == 0:
                #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
                #    print(nms_boxes)

                for nms_box in nms_boxes:
                    all_pred_boxes.append([train_idx] + nms_box)

                for box in true_bboxes[idx]:
                    # many will get converted to 0 pred, as bboxes have Conf = 1 and the rest are 0
                    if box[1] > score_thres:
                        all_true_boxes.append([train_idx] + box)

                train_idx += 1

    #model.train()
    return all_pred_boxes, all_true_boxes

# Function to Print mAP metrics

In [14]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    print('Class F1-Score')
    smoke_f1 = 2 * (cls_prec[0] * cls_rec[0]) / (cls_prec[0] + cls_rec[0])
    fire_f1 = 2 * (cls_prec[1] * cls_rec[1]) / (cls_prec[1] + cls_rec[1])
    print(f'- {smoke:<10}' + f'{smoke_f1:.4f}')
    print(f'- {fire:<10}' + f'{fire_f1:.4f}')

### Log Path

In [15]:
log_path = './mAP_&_pr_curves/classify_&_detect/'

# Brevitas No Comp

## Score Thres = 0.2

In [16]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = brevitas_no_comp_name,
    score_thres = 0.2
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [09:09<00:00,  1.52s/it]


In [17]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.22s/it]


In [18]:
print_metrics(score_02_metrics)

mAP @0.50   0.6135
Average Precision
- Smoke     0.6210
- Fire      0.6059
Class Precision
- Smoke     0.6978
- Fire      0.7003
Class Recall
- Smoke     0.6822
- Fire      0.6503
Class F1-Score
- Smoke     0.6899
- Fire      0.6744


## Score Thres = 0.001

In [19]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = brevitas_no_comp_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [28:44<00:00,  4.78s/it]


In [20]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:56<00:00, 208.50s/it]


In [21]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6446
Average Precision
- Smoke     0.6656
- Fire      0.6236
Class Precision
- Smoke     0.0124
- Fire      0.0796
Class Recall
- Smoke     0.8030
- Fire      0.6845
Class F1-Score
- Smoke     0.0244
- Fire      0.1426


In [22]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_dfire_loader,
    detector_model_name = brevitas_no_comp_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [05:14<00:00,  5.07s/it]


In [23]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.96s/it]


In [24]:
print_metrics(score_0001_metrics)

mAP @0.50   0.3959
Average Precision
- Smoke     0.4562
- Fire      0.3357
Class Precision
- Smoke     0.0097
- Fire      0.0744
Class Recall
- Smoke     0.6921
- Fire      0.4585
Class F1-Score
- Smoke     0.0191
- Fire      0.1280


In [25]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_uav_loader,
    detector_model_name = brevitas_no_comp_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 57/57 [04:39<00:00,  4.91s/it]


In [26]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.82s/it]


In [27]:
print_metrics(score_0001_metrics)

mAP @0.50   0.5879
Average Precision
- Smoke     0.6621
- Fire      0.5138
Class Precision
- Smoke     0.0122
- Fire      0.1110
Class Recall
- Smoke     0.7258
- Fire      0.5327
Class F1-Score
- Smoke     0.0240
- Fire      0.1838


In [28]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_cv_loader,
    detector_model_name = brevitas_no_comp_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 242/242 [18:55<00:00,  4.69s/it]


In [29]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [03:23<00:00, 101.67s/it]


In [30]:
print_metrics(score_0001_metrics)

mAP @0.50   0.7088
Average Precision
- Smoke     0.7073
- Fire      0.7102
Class Precision
- Smoke     0.0132
- Fire      0.0765
Class Recall
- Smoke     0.8514
- Fire      0.7738
Class F1-Score
- Smoke     0.0260
- Fire      0.1393


# Brevitas AIMET FPGA 

## Score Thres = 0.2

In [31]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = brevitas_aimet_name,
    score_thres = 0.2
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [09:03<00:00,  1.51s/it]


In [32]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.68s/it]


In [33]:
print_metrics(score_02_metrics)

mAP @0.50   0.6140
Average Precision
- Smoke     0.6145
- Fire      0.6136
Class Precision
- Smoke     0.6760
- Fire      0.6233
Class Recall
- Smoke     0.6792
- Fire      0.6647
Class F1-Score
- Smoke     0.6776
- Fire      0.6433


## Score Thres = 0.001

## Full DS

In [34]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = brevitas_aimet_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [21:02<00:00,  3.50s/it]


In [35]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [05:41<00:00, 170.80s/it]


In [36]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6409
Average Precision
- Smoke     0.6557
- Fire      0.6260
Class Precision
- Smoke     0.0162
- Fire      0.0746
Class Recall
- Smoke     0.7904
- Fire      0.6913
Class F1-Score
- Smoke     0.0317
- Fire      0.1347


## DFire 

In [37]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_dfire_loader,
    detector_model_name = brevitas_aimet_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [03:56<00:00,  3.82s/it]


In [38]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.82s/it]


In [39]:
print_metrics(score_0001_metrics)

mAP @0.50   0.4006
Average Precision
- Smoke     0.4511
- Fire      0.3501
Class Precision
- Smoke     0.0121
- Fire      0.0695
Class Recall
- Smoke     0.6839
- Fire      0.4806
Class F1-Score
- Smoke     0.0237
- Fire      0.1214


## FASDD UAV

In [40]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_uav_loader,
    detector_model_name = brevitas_aimet_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 57/57 [02:56<00:00,  3.09s/it]


In [41]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.22s/it]


In [42]:
print_metrics(score_0001_metrics)

mAP @0.50   0.5851
Average Precision
- Smoke     0.6472
- Fire      0.5230
Class Precision
- Smoke     0.0184
- Fire      0.1157
Class Recall
- Smoke     0.7176
- Fire      0.5415
Class F1-Score
- Smoke     0.0359
- Fire      0.1907


## FASDD CV

In [43]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_cv_loader,
    detector_model_name = brevitas_aimet_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 242/242 [14:08<00:00,  3.51s/it]


In [44]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:46<00:00, 83.26s/it]


In [45]:
print_metrics(score_0001_metrics)

mAP @0.50   0.7032
Average Precision
- Smoke     0.6985
- Fire      0.7079
Class Precision
- Smoke     0.0168
- Fire      0.0709
Class Recall
- Smoke     0.8367
- Fire      0.7765
Class F1-Score
- Smoke     0.0330
- Fire      0.1300


# Brevitas AIMET CPU

## Score Thres = 0.2

In [46]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = brevitas_aimet_cpu_name,
    score_thres = 0.2
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [09:04<00:00,  1.51s/it]


In [47]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.82s/it]


In [48]:
print_metrics(score_02_metrics)

mAP @0.50   0.6317
Average Precision
- Smoke     0.6485
- Fire      0.6148
Class Precision
- Smoke     0.6821
- Fire      0.6284
Class Recall
- Smoke     0.7063
- Fire      0.6697
Class F1-Score
- Smoke     0.6940
- Fire      0.6484


## Score Thres = 0.001

## Full DS

In [49]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = brevitas_aimet_cpu_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 361/361 [22:54<00:00,  3.81s/it]


In [50]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:54<00:00, 207.13s/it]


In [51]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6572
Average Precision
- Smoke     0.6874
- Fire      0.6269
Class Precision
- Smoke     0.0154
- Fire      0.0712
Class Recall
- Smoke     0.8105
- Fire      0.6969
Class F1-Score
- Smoke     0.0303
- Fire      0.1293


## Dfire

In [52]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_dfire_loader,
    detector_model_name = brevitas_aimet_cpu_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [04:09<00:00,  4.02s/it]


In [53]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.13s/it]


In [54]:
print_metrics(score_0001_metrics)

mAP @0.50   0.4154
Average Precision
- Smoke     0.4826
- Fire      0.3483
Class Precision
- Smoke     0.0120
- Fire      0.0659
Class Recall
- Smoke     0.7014
- Fire      0.4813
Class F1-Score
- Smoke     0.0236
- Fire      0.1159


## FASDD UAV

In [55]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_uav_loader,
    detector_model_name = brevitas_aimet_cpu_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 57/57 [03:27<00:00,  3.64s/it]


In [56]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.74s/it]


In [57]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6161
Average Precision
- Smoke     0.6839
- Fire      0.5483
Class Precision
- Smoke     0.0165
- Fire      0.1187
Class Recall
- Smoke     0.7454
- Fire      0.5651
Class F1-Score
- Smoke     0.0323
- Fire      0.1962


## FASDD CV

In [58]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_fasdd_cv_loader,
    detector_model_name = brevitas_aimet_cpu_name,
    score_thres = 0.001
)

Get Boxes: 100%|█████████████████████████████████████████████████████████████████████████████████| 242/242 [15:18<00:00,  3.80s/it]


In [59]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [02:59<00:00, 89.77s/it]


In [60]:
print_metrics(score_0001_metrics)

mAP @0.50   0.7169
Average Precision
- Smoke     0.7292
- Fire      0.7047
Class Precision
- Smoke     0.0161
- Fire      0.0671
Class Recall
- Smoke     0.8553
- Fire      0.7789
Class F1-Score
- Smoke     0.0316
- Fire      0.1236
